In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex Pipelines: Vertex AI Custom Job training wrapper

## Overview
This notebook shows how to use the `run_as_vertex_ai_custom_job` wrapper to convert any component to run it as a Vertex AI custom job. This allows users to take advantage of the vertical and horizontal scaling for computation heavy tasks on Vertex AI. This requires that the underlying component has built in support distributed computation. To learn more about Vertex AI Custom Job see [Vertex AI Custom Training](https://cloud.google.com/vertex-ai/docs/training/custom-training). 


### Objective

In this example, you'll learn how to use `run_as_vertex_ai_custom_job` to:
- Convert a component spec to a Vertex AI custom training job component
- Create a pipeline using the component
- Submit the job to Managed Pipeline.

The components are [documented here](https://google-cloud-pipeline-components.readthedocs.io/en/latest/google_cloud_pipeline_components.aiplatform.html#module-google_cloud_pipeline_components.aiplatform).


### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI Training and Serving
* Cloud Storage

Learn about pricing for [Vertex AI](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


## Vertex AI Custom Job Wrapper Interface

`google_cloud_pipeline_components.experimental.custom_job.custom_job`

```python
def run_as_vertex_ai_custom_job(
    component_spec: Callable,
    display_name: Optional[str] = None,
    replica_count: Optional[int] = None,
    machine_type: Optional[str] = None,
    accelerator_type: Optional[str] = None,
    accelerator_count: Optional[int] = None,
    boot_disk_type: Optional[str] = None,
    boot_disk_size_gb: Optional[int] = None,
    timeout: Optional[str] = None,
    restart_job_on_worker_restart: Optional[bool] = None,
    service_account: Optional[str] = None,
    network: Optional[str] = None,
    worker_pool_specs: Optional[List[Mapping[str, Any]]] = None,
) -> Callable:
```

<dt><b>Args:</b></dt><dd><p>component_spec: The component spec (factory function) to run as aiplatform custom job.
display_name: Optional. The name of the custom job. If not provided the</p>
<div><p>component_spec.name will be used instead.</p>
</div>
<dl class="simple">
<dt>replica_count: Optional. The number of replicas to be split between master</dt><dd><p>workerPoolSpec and worker workerPoolSpec. (master always has 1 replica).</p>
</dd>
<dt>machine_type: Optional. The type of the machine to run the custom job. The</dt><dd><p>default value is “n1-standard-4”.</p>
</dd>
<dt>accelerator_type: Optional. The type of accelerator(s) that may be attached</dt><dd><p>to the machine as per accelerator_count. Optional.</p>
</dd>
<dt>accelerator_count: Optional. The number of accelerators to attach to the</dt><dd><p>machine.</p>
</dd>
<dt>boot_disk_type: Optional. Type of the boot disk (default is “pd-ssd”). Valid</dt><dd><dl class="simple">
<dt>values: “pd-ssd” (Persistent Disk Solid State Drive) or “pd-standard”</dt><dd><p>(Persistent Disk Hard Disk Drive).</p>
</dd>
</dl>
</dd>
</dl>
<p>boot_disk_size_gb: Optional. Size in GB of the boot disk (default is 100GB).
timeout: Optional. The maximum job running time. The default is 7 days. A</p>
<div><p>duration in seconds with up to nine fractional digits, terminated by ‘s’.
Example: “3.5s”</p>
</div>
<dl class="simple">
<dt>restart_job_on_worker_restart: Optional. Restarts the entire CustomJob if a</dt><dd><p>worker gets restarted. This feature can be used by distributed training
jobs that are not resilient to workers leaving and joining a job.</p>
</dd>
<dt>service_account: Optional. Specifies the service account for workload run-as</dt><dd><p>account.</p>
</dd>
<dt>network: Optional. The full name of the Compute Engine network to which the</dt><dd><p>job should be peered. For example, projects/12345/global/networks/myVPC.</p>
</dd>
<dt>worker_pool_specs: Optional, worker_pool_specs for distributed training. this</dt><dd><p>will overwite all other cluster configurations. For details, please see:
<a class="reference external" href="https://cloud.google.com/ai-platform-unified/docs/training/distributed-training">https://cloud.google.com/ai-platform-unified/docs/training/distributed-training</a></p>
</dd>
</dl>
</dd>
<dt><b>Returns:</b></dt><dd><p>A Custom Job component OP correspoinding to the input component OP.</p>
</dd>
</dl>
</dd></dl>

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
!pip3 install {USER_FLAG}  -U "git+https://github.com/kubeflow/pipelines.git#egg=kfp&subdirectory=sdk/python" -q
!pip3 install {USER_FLAG}  -U google-cloud-pipeline-components -q
!pip3 install {USER_FLAG}  -U google-cloud-aiplatform -q

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin
### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket as necessary

You need a Cloud Storage bucket for this example.  If you don't have one that you want to use, you can make one now.


Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex Pipelines.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil ls -al $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants

In [ ]:
import google_cloud_pipeline_components as gcpc
from kfp.v2 import components, compiler, dsl

In [ ]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

USER = "your-user-name"  # <---CHANGE THIS
PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET_NAME, USER)

PIPELINE_ROOT

## Create a component, convert it to a custom job to use in a pipeline.
Create a simple component that takes an input and produces an output

In [ ]:
producer_op = components.load_component_from_text(
    """
name: Producer
inputs:
- {name: input_text, type: String, description: 'Represents an input parameter.'}
outputs:
- {name: output_value, type: String, description: 'Represents an output paramter.'}
implementation:
  container:
    image: google/cloud-sdk:latest
    command:
    - sh
    - -c
    - |
      set -e -x
      echo "$0, this is an output parameter" | gsutil cp - "$1"
    - {inputValue: input_text}
    - {outputPath: output_value}
"""
)

Convert the component to a Vertex AI Custom Job

In [ ]:
# This is a temporary work around for v0.1.4 to point to the correct base image for custom_job
gcpc.experimental.custom_job.custom_job._DEFAULT_CUSTOM_JOB_CONTAINER_IMAGE = 'gcr.io/ml-pipeline/gcp-launcher:0.1.4'

custom_job_producer_op= gcpc.experimental.custom_job.custom_job.run_as_vertex_ai_custom_job(producer_op)

Define the pipeline:

In [ ]:
@dsl.pipeline(pipeline_root='', name='custom-job-sample-pipeline')
def pipeline(text: str = 'message'):
    custom_producer_task = custom_job_producer_op(input_text=text, gcp_project=PROJECT_ID ,gcp_region=REGION)

## Compile and run the pipeline

Compile the pipeline:

In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="simple_custom_job_spec.json"
)

The pipeline compilation generates the `simple_custom_job_spec.json` job spec file.

Next, instantiate an API client object and run the defined pipeline like this: 

In [ ]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(project_id=PROJECT_ID, region=REGION)

response = api_client.create_run_from_job_spec(
          job_spec_path='simple_custom_job_spec.json',
          pipeline_root=PIPELINE_ROOT,  enable_caching=False
          )

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:
- Delete Cloud Storage objects that were created.  Uncomment and run the command in the cell below **only if you are not using the `PIPELINE_ROOT` path for any other purpose**.
- Delete your deployed model: first, undeploy it from its *endpoint*, then delete the model and endpoint.


In [ ]:
# Warning: this command will delete ALL Cloud Storage objects under the PIPELINE_ROOT path.
# ! gsutil -m rm -r $PIPELINE_ROOT